# Using Neural Networks to Predict Oil Peak Rate

### Loading and preparing the data
For a neural network, the data is best normalized using mean / standard deviation normalization. However, for the positional data, to avoid the warping of space, they both have the same standard deviation for the sake of normalization.

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/drive')
# Load the training and testing data
df = pd.read_csv('/content/drive/MyDrive/Datathon 2024/Data/preprocessed_training.csv')

# Gets the testing and training indices from the csv in the drive, shared with XGBoost
training_indices = np.array(pd.read_csv('/content/drive/MyDrive/Datathon 2024/Data/StandardSplit.csv')["train_test_split"])
training_indices -= 1

# Dropping columns that are either redundant, very hard to represent, or irrelevant
df.drop(["standardized_operator_name", "proppant_to_frac_fluid_ratio", "total_proppant", "total_fluid", "horizontal_dist_x", "horizontal_dist_y"], inplace=True, axis=1)

# Saving the target variable standard deviation and mean before normalization to convert the data back later
oil_std = df["OilPeakRate"].std()
oil_mean = df["OilPeakRate"].mean()

# Creating a standard deviation value for both the x and y coordinates as to not cause warping during normalization
coord_std = (df["surface_x"].std() + df["surface_y"].std()) / 2

# Mean standard deviation normalization function
def mean_std_normalization(column):
    mean = column.mean()
    std_dev = column.std()
    if column.name not in ("surface_x", "surface_y"):
      normalized_column = (column - mean) / std_dev
    else:
      normalized_column = (column - mean) / coord_std
    return normalized_column

# Apply mean standard deviation normalization to each column
df = df.apply(mean_std_normalization)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,OilPeakRate,surface_x,surface_y,gross_perforated_length,true_vertical_depth,proppant_intensity,frac_fluid_intensity,bin_lateral_length,frac_seasoning,horizontal_dist,...,Undefined,Inner Well,Outer Well,Batch-Concurrent Frac,Batch-Sequential Frac,Non-Batch Frac,Infill Child Well,Sibling Well,Standalone Well,Unknown
0,-1.112048,1.473690,-1.489933,-1.004684,-0.064585,-1.685204,-1.561580,-0.779946,-0.463977,-0.969622,...,-0.233983,-0.571134,-0.668564,-0.560283,-0.412411,-0.757834,-0.485485,-0.756733,1.171525,-0.155543
1,-1.027789,1.460921,-1.511878,-1.334935,-0.110165,-1.376399,-1.323028,-0.779946,-0.159840,-1.267642,...,-0.233983,-0.571134,1.495665,-0.560283,-0.412411,-0.757834,2.059687,-0.756733,-0.853544,-0.155543
2,-1.342079,1.459185,-1.487842,-1.069325,-0.016885,-0.628038,-0.481106,-0.779946,0.509263,-1.082754,...,-0.233983,-0.571134,-0.668564,-0.560283,-0.412411,1.319483,-0.485485,-0.756733,1.171525,-0.155543
3,-0.616681,1.473989,-1.533268,-1.105790,-0.105925,-1.505428,-1.522248,-0.779946,-0.366653,-0.998418,...,-0.233983,-0.571134,-0.668564,-0.560283,-0.412411,-0.757834,-0.485485,-0.756733,1.171525,-0.155543
4,-0.754609,1.455320,-1.549551,-1.068911,-0.090555,-1.488616,-1.367058,-0.779946,-0.354488,-0.976812,...,-0.233983,-0.571134,1.495665,-0.560283,-0.412411,-0.757834,2.059687,-0.756733,-0.853544,-0.155543


## Adding the Gaussian Approximations
An approximation of the oil peak rate value can be acquired by looking at the surrounding points. For each training and testing point, a Gaussian probability density function (PDF) is created. Then, the surrounding training wells will have their peak oil values multiplied by the PDF's value at the well's location. All of these products are summed and divided by the sum of all PDF values used. This is the approximation.

It is important to note that data leaks are avoided because only TRAINING wells are searched for in the surrounding area.

In [2]:
from scipy.spatial import cKDTree
from scipy.stats import norm

QUERY_RADIUS = 0.3 # Radius (in xy-coordinate standard deviations defined further above) at which the estimator will query for training points
STDEV = 0.01 # The standard deviation of the Gaussian distribution itself

def add_gaussian_approximations(train_tensor, train_y, test_tensor):
  """
  Returns the updated training tensors with a new column including the
  Gaussian estimates for each point
  """
  train_tensor, test_tensor, train_y = train_tensor.numpy(), test_tensor.numpy(), train_y.numpy()

  # Getting the xy points
  points = np.array([train_tensor[:,0], train_tensor[:,1]]).transpose()

  # Forming it into a KD tree for fast querying in the long run, very easy to search for nearby points
  kdtree = cKDTree(points)

  # Creating a lookup table for the oil values for each point
  data_dict = {}
  for i, row in enumerate(train_tensor):
    if row[0] not in data_dict:
      data_dict[row[0]] = {}
    data_dict[row[0]][row[1]] = train_y[i,0]

  def gaussian_pdf(x, mean, std_dev):
    """
    Returns the value of the Gaussian PDF at x with a distribution with a given mean and standard deviation
    """
    exponent = -0.5 * ((x - mean) / std_dev) ** 2
    pdf = (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(exponent)
    return pdf

  def get_gaussian(point):
    """
    Gets the Gaussian estimate for a single point using the Gaussian PDF
    """
    point = np.array(point)

    # Queries for surrounding points
    other_points = points[kdtree.query_ball_point(point, QUERY_RADIUS)]

    if len(other_points) < 10:
      # If there is nothing in the query neighborhood, just use the entire training set. Should be rare so not much of a slowdown expected.
      other_points = points

    # Gets the list of oil values for each point
    vals = np.array([data_dict[pt[0]][pt[1]] for pt in other_points])

    # Calculates the Gaussian PDF value based on the distance from the point for each nearby training well
    distances = np.linalg.norm(other_points - point, axis=1)
    gaussians = gaussian_pdf(distances, 0, STDEV)
    gaussians, vals = gaussians[distances != 0], vals[distances != 0]

    # Adds the products and normalizes for the final estimate
    result = sum(vals * gaussians) / sum(gaussians)
    return result

  # Handles nan issues, adds the Gaussian column to the train and test data sets.
  train_results = torch.tensor(np.nan_to_num(np.array([get_gaussian([row[0], row[1]]) for row in torch.tensor(train_tensor)]).reshape(-1,1), nan=0))
  train_results = torch.cat((torch.tensor(train_tensor), train_results), dim=1)
  test_results = torch.tensor(np.nan_to_num(np.array([get_gaussian([row[0], row[1]]) for row in torch.tensor(test_tensor)]).reshape(-1,1), nan=0))
  test_results = torch.cat((torch.tensor(test_tensor), test_results), dim=1)
  return train_results, test_results

Now that the functions have been defined, we can actually create our test and train variables

In [3]:
import torch
from sklearn.model_selection import train_test_split

# Creates the X and y data sets, where the OilPeakRate is the target variable
X = df[df.drop(["OilPeakRate"], axis=1).columns]
y = df["OilPeakRate"]

# Converts them into tensors to be used in the neural network
X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# Splits into training and testing data based on the indices specified in the csv file at the top of the code
train_indices = np.setdiff1d(np.arange(len(df)), training_indices)
X_test, X_train = X_tensor[train_indices], X_tensor[training_indices]
y_test, y_train = y_tensor[train_indices], y_tensor[training_indices]

# Adds the Gaussian approximations to the X data
X_train, X_test = add_gaussian_approximations(X_train, y_train, X_test)

<ipython-input-2-ac1cf4c84714>:57: RuntimeWarning: invalid value encountered in double_scalars
  result = sum(vals * gaussians) / sum(gaussians)


In [4]:
# Validate that there is a correlation between the ground truth y data and the Gaussian estimation ( > 0.6 is amazing compared to other data points alone)
np.corrcoef(X_test[:,-1].numpy().flatten(), y_test.numpy().flatten())[0,1]

0.6598116659699832

## Neural Network Setup
The data is split into three categories:
*   Well info: categorical variables that describe the type and construction of the well
*   Location: the xy coordinates of the well
*   Structure: the length or amount of the important parts of the well

Then it is brought through the following layers
*   Dense layer for each category
*   Concatenating the results
*   A sigmoid activation layer to introduce non-linearity
*   A dense layer
*   A ReLU activation layer
*   A dense layer

This allows for each type of data to be encoded first with other pieces of data that are likely relevant to it before being concatenated and fully integrated with a larger dense layer.



In [5]:
# Split the data into three categories
well_info_train, well_info_test = X_train[:,-15:], X_test[:,-15:]
location_train, location_test = X_train[:,:2], X_test[:,:2]
structure_train, structure_test = X_train[:,2:-15], X_test[:,2:-15]

In [6]:
import torch.nn as nn

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, well_info_size, location_size, structure_size):
        super(NeuralNetwork, self).__init__()
        self.f1 = nn.Linear(well_info_size, 3) # Each category has a number of output channels (this has been experimented with)
        self.f2 = nn.Linear(location_size, 6)
        self.f3 = nn.Linear(structure_size, 7)
        self.relu = nn.Sigmoid()
        self.fc2 = nn.Linear(16, 32) # Each dense layer can increase or decrease in dimension. This one goes from 16 to 32
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1) # This dense layer creates the output variable we are trying to predict.

    def forward(self, well_info_x, location_x, structure_x):
        x1 = self.f1(well_info_x)
        x2 = self.f2(location_x)
        x3 = self.f3(structure_x)
        x = self.relu(torch.cat((x1, x2, x3), dim=1)) # This is where the concatenation of the category outputs from their small dense layers happen
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

## Neural Network Training
This is trained with...
*   6000 epochs
*   Optimizer: Stochastic Gradient Descent (SGD)
*   Learning Rate: 0.05
*   Momentum: .001



In [7]:
import torch.optim as optim
X_train, X_test = X_train.double(), X_test.double()
y_train, y_test = y_train.double(), y_test.double()

# Initialize the model
well_info_size, location_size, structure_size = well_info_train.shape[1], location_train.shape[1], structure_train.shape[1]
model = NeuralNetwork(well_info_size, location_size, structure_size).double()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=1e-3)


In [8]:
# Training the model
num_epochs = 6000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(well_info_train, location_train, structure_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 300 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        with torch.no_grad():
          test_outputs = model(well_info_test, location_test, structure_test)
          test_loss = criterion(test_outputs, y_test)
          print(f'Test Loss: {test_loss.item():.4f}')


Epoch [300/6000], Loss: 0.5519
Test Loss: 0.5193
Epoch [600/6000], Loss: 0.4749
Test Loss: 0.4458
Epoch [900/6000], Loss: 0.4633
Test Loss: 0.4356
Epoch [1200/6000], Loss: 0.4594
Test Loss: 0.4319
Epoch [1500/6000], Loss: 0.4566
Test Loss: 0.4292
Epoch [1800/6000], Loss: 0.4543
Test Loss: 0.4267
Epoch [2100/6000], Loss: 0.4522
Test Loss: 0.4244
Epoch [2400/6000], Loss: 0.4501
Test Loss: 0.4225
Epoch [2700/6000], Loss: 0.4483
Test Loss: 0.4209
Epoch [3000/6000], Loss: 0.4468
Test Loss: 0.4196
Epoch [3300/6000], Loss: 0.4453
Test Loss: 0.4185
Epoch [3600/6000], Loss: 0.4441
Test Loss: 0.4177
Epoch [3900/6000], Loss: 0.4429
Test Loss: 0.4170
Epoch [4200/6000], Loss: 0.4419
Test Loss: 0.4165
Epoch [4500/6000], Loss: 0.4411
Test Loss: 0.4161
Epoch [4800/6000], Loss: 0.4403
Test Loss: 0.4156
Epoch [5100/6000], Loss: 0.4396
Test Loss: 0.4153
Epoch [5400/6000], Loss: 0.4390
Test Loss: 0.4150
Epoch [5700/6000], Loss: 0.4388
Test Loss: 0.4149
Epoch [6000/6000], Loss: 0.4391
Test Loss: 0.4154


In [9]:
model.eval()

NeuralNetwork(
  (f1): Linear(in_features=15, out_features=3, bias=True)
  (f2): Linear(in_features=2, out_features=6, bias=True)
  (f3): Linear(in_features=8, out_features=7, bias=True)
  (relu): Sigmoid()
  (fc2): Linear(in_features=16, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)

## Reviewing the testing data predictions and converting to a CSV

In [10]:
# Displays the RMSE
(np.mean(((model(well_info_test, location_test, structure_test).detach().numpy() - y_test.detach().numpy()) * oil_std) ** 2) ** (1/2))

100.40882511548143

In [11]:
# Reconstructs the real oil time data and converts it to a csv file to be used with XGBoost
preds = model(well_info_test, location_test, structure_test).detach().numpy().transpose()[0] * oil_std + oil_mean
preds = pd.Series(preds)
preds.name = "predictions"
preds.to_csv("/content/drive/MyDrive/Datathon 2024/Data/neural_network_prediction.csv", index=False)